In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
#!wget -q http://apache.osuosl.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz

#Per velocizzare il tutto si potrebbe eseguire il download del file a parte e poi quando bisogna usarlo, fare mount del gdrive e unzip solamente.

# unzip it
!tar xf /content/drive/MyDrive/spark-3.4.2-bin-hadoop3.tgz
# install findspark
!pip install -q findspark
!pip install fuzzywuzzy

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"

In [5]:
import findspark
findspark.init()

In [6]:
# Verify the Spark version running on the virtual cluster
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

assert  "3." in sc.version, "Verify that the cluster Spark's version is 3.x"

In [18]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, explode, collect_list, udf
from pyspark.sql.types import StringType
import pandas as pd
from fuzzywuzzy import process, fuzz

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
spark = SparkSession(sc)
print(spark)

In [95]:
# Funzione utile a verificare se il nome di un'entità di tipo Person è contenuto all'interno di un'altra entità
def create_name_mapping(names):
    name_mapping = {}
    for name in names:
        for other_name in names:
            if (name != other_name and name.lower() in other_name.lower()) or (name != other_name and name.lower() == other_name.lower()):
                name_mapping[name] = other_name
                break
        if name not in name_mapping:
            name_mapping[name] = name
    return name_mapping

# Funzione per trovare i nomi completi degli acronimi
def find_full_name(name, mapping_dict):
    if name is not None:
        return mapping_dict.get(name.lower(), name)
    return name


@udf(StringType())
def replace_acronyms(name):
    return mapping_dict.get(name.lower(), name)

def find_canonical_name(name, names_list, threshold=85):
    match = process.extractOne(name, names_list, scorer=fuzz.token_sort_ratio)
    if match and match[1] >= threshold:
        return match[0]
    return name

@udf(StringType())
def map_names(name):
    return name_map.get(name, name)

In [60]:
# Liste di verifica
entities_list = ["Person", "Organization", "Location", "Politician", "Party", "Event", "Agreement"]
relationship_list = ["leader_of", "is_from", "part_of", "located_in", "member_of", "supports", "president_of", "opposition", "colleague"]

# Carica i file CSV in DataFrame Spark
df_entities = spark.read.csv("/content/drive/MyDrive/FinalProject/entities.csv", header=True)
df_relations = spark.read.csv("/content/drive/MyDrive/FinalProject/relations.csv", header=True)
df_acronimi = spark.read.csv("/content/drive/MyDrive/FinalProject/check_entities.csv", header=True)

# Elimina duplicati e filtra per tipi consentiti
df_entities = df_entities.withColumn('Entity_lower', lower(col('Name')))\
                          .dropDuplicates(['Entity_lower', 'Type'])\
                          .filter(col('Type').isin(entities_list))\
                          .drop('Entity_lower')

df_relations = df_relations.dropDuplicates(['Relation', 'Source', 'Target'])\
                           .filter(col('Relation').isin(relationship_list))

In [67]:
mapping_dict = dict(df_acronimi.rdd.map(lambda row: (row['Old_Name'].lower(), row['New_Name'])).collect())

In [69]:
df_entities = df_entities.withColumn("Name", replace_acronyms(col("Name")))
df_relations = df_relations.withColumn("Source", replace_acronyms(col("Source")))
df_relations = df_relations.withColumn("Target", replace_acronyms(col("Target")))

In [71]:
unique_names = df_entities.select("Name").distinct().rdd.flatMap(lambda x: x).collect()
name_map = {name: find_canonical_name(name, unique_names) for name in unique_names}

In [96]:
df_person = df_entities.select("Name").filter(col("Type") == "Person").dropDuplicates()
names = [row["Name"] for row in df_person.collect()]
name_map_person = create_name_mapping(names)

In [98]:
name_map.update(name_map_person)

In [100]:
df_entities = df_entities.withColumn("Name", map_names(col("Name")))
df_relations = df_relations.withColumn("Source", map_names(col("Source")))
df_relations = df_relations.withColumn("Target", map_names(col("Target")))

In [149]:
df_unique_entities = df_entities.dropDuplicates()
df_unique_relations = df_relations.dropDuplicates()

In [150]:
df_unique_entities = df_unique_entities.toPandas()
df_unique_relations = df_unique_relations.toPandas()

df_unique_entities.to_csv("/content/drive/MyDrive/FinalProject/unique_entities.csv", index=False)
df_unique_relations.to_csv("/content/drive/MyDrive/FinalProject/unique_relations.csv", index=False)

print("File con righe uniche (nomi simili gestiti) salvati come unique_entities.csv e unique_relations.csv")

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "<ipython-input-68-1169a7b7f0a8>", line 29, in replace_acronyms
AttributeError: 'NoneType' object has no attribute 'lower'


In [ ]:
spark.stop()